In [1]:
import argparse
import os
import sys

# Simulate having cfg available by loading in hydra config as dict
import yaml
try:
    from yaml import CLoader as Loader, CDumper as Dumper
except ImportError:
    from yaml import Loader, Dumper

import dgl
import dgl.function as fn
import hydra
import numpy as np
import torch
import torch.nn as nn
import torch.nn.functional as F
from torchmetrics import MeanMetric
from pytorch_lightning import LightningDataModule, LightningModule, Trainer
from pytorch_lightning.callbacks import ModelCheckpoint

user_net_id = os.getlogin()
home_path = '/scratch/' + user_net_id + '/projects/NYU-Zillow-Capstone-2022-Team-A'
if home_path not in sys.path:
    sys.path.append('/scratch/' + user_net_id + '/projects/NYU-Zillow-Capstone-2022-Team-A')

from src.datamodules.negative_sampler import NegativeSampler
from src.model.SAGE import SAGE


/ext3/conda/zillow_MMKG/lib/python3.8/site-packages/tqdm/auto.py:22: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [2]:
# Simulate having cfg available by loading in hydra config as dict
from types import SimpleNamespace

class NestedNamespace(SimpleNamespace):
    def __init__(self, dictionary, **kwargs):
        super().__init__(**kwargs)
        for key, value in dictionary.items():
            if isinstance(value, dict):
                self.__setattr__(key, NestedNamespace(value))
            else:
                self.__setattr__(key, value)

cfg = NestedNamespace(yaml.load(open('../conf/config.yaml'), Loader=Loader))

import json
csv_dataset_root = '../' + cfg.data.zillow_root
modal_node_ids = json.load(open(f'{csv_dataset_root}/modal_node_ids.json', 'r'))

## NOTEBOOK ORGANIZATION:
This notebook is for exploring and understanding train_graphsage.py. Given that main() runs train(cfg), where cfg is passed from Hydra, we can explore the notebook by deep diving into each function, class, and procedure that goes into train(). The train() function is provided below, and subsequent cells investigate intermediate outputs of its function calls and other important code blocks.

## Step 1: Exploring DataModule Class

Below we show the DataModule class definition and dependencies, then execute lines of DataModule, step by step, and examine outputs to understand intermediate values and what's going on overall.

In [3]:
def to_bidirected_with_reverse_mapping(g):
    """Makes a graph bidirectional, and returns a mapping array ``mapping`` where ``mapping[i]``
    is the reverse edge of edge ID ``i``. Does not work with graphs that have self-loops.
    """
    g_simple, mapping = dgl.to_simple(
        dgl.add_reverse_edges(g), return_counts="count", writeback_mapping=True
    )
    c = g_simple.edata["count"]
    num_edges = g.num_edges()
    mapping_offset = torch.zeros(g_simple.num_edges() + 1, dtype=g_simple.idtype)
    mapping_offset[1:] = c.cumsum(0)
    idx = mapping.argsort()
    idx_uniq = idx[mapping_offset[:-1]]
    reverse_idx = torch.where(
        idx_uniq >= num_edges, idx_uniq - num_edges, idx_uniq + num_edges
    )
    reverse_mapping = mapping[reverse_idx]
    # sanity check
    src1, dst1 = g_simple.edges()
    src2, dst2 = g_simple.find_edges(reverse_mapping)
    assert torch.equal(src1, dst2)
    assert torch.equal(src2, dst1)
    return g_simple, reverse_mapping

class NegativeSamplerTest(object):
    def __init__(self, g, k, max_img_id, neg_share=False):
        self.weights = g.in_degrees().float() ** 0.75
        self.k = k
        self.neg_share = neg_share
        self.max_img_id = max_img_id

    def __call__(self, g, eids):
        src, _ = g.find_edges(eids)
        img_node_mask = src <= self.max_img_id
        src = src[img_node_mask]
        
        n = len(src)
        if self.neg_share and n % self.k == 0:
            dst = self.weights.multinomial(n, replacement=True)
            dst = dst.view(-1, 1, self.k).expand(-1, self.k, -1).flatten()
        else:
            dst = self.weights.multinomial(n * self.k, replacement=True)
        src = src.repeat_interleave(self.k)
        return src, dst

class DataModule(LightningDataModule):
    def __init__(
        self,
        csv_dataset_root,
        modal_node_ids_file,
        data_cpu=False,
        fan_out=[10, 25],
        device="cpu",
        batch_size=1024,
        num_workers=4,
        force_reload=False,
    ):
        super().__init__()
        self.save_hyperparameters()
        dataset = dgl.data.CSVDataset(csv_dataset_root, force_reload=force_reload)
        g = dataset[0]
        g_bid, reverse_eids = to_bidirected_with_reverse_mapping(g)
        # g = g.formats(["csc"])
        g_bid = g_bid.to(device)
        g = g.to(device)
        reverse_eids = reverse_eids.to(device)
        # seed_edges = torch.arange(g.num_edges()).to(device)

        max_img_id = max(json.load(open(modal_node_ids_file, 'r'))['images'])

        train_nid = torch.nonzero(g_bid.ndata["train_mask"], as_tuple=True)[0].to(device)
        val_nid = torch.nonzero(g_bid.ndata["val_mask"], as_tuple=True)[0].to(device)
        test_nid = torch.nonzero(
            ~(g_bid.ndata["train_mask"] | g_bid.ndata["val_mask"]), as_tuple=True
        )[0].to(device)

        sampler = dgl.dataloading.MultiLayerNeighborSampler(
            [int(_) for _ in fan_out], prefetch_node_feats=["feat"]
        )

        self.g = g
        self.g_bid = g_bid
        self.train_nid, self.val_nid, self.test_nid = train_nid, val_nid, test_nid
        self.sampler = sampler
        self.device = device
        self.batch_size = batch_size
        self.num_workers = num_workers
        self.in_dim = g_bid.ndata["feat"].shape[1]
        self.reverse_eids = reverse_eids
        self.max_img_id = max_img_id

    def train_dataloader(self):
        sampler = dgl.dataloading.as_edge_prediction_sampler(
            self.sampler,
            exclude="reverse_id",
            reverse_eids=self.reverse_eids,
            negative_sampler=NegativeSampler(self.g, 5, self.max_img_id)
            # negative_sampler=dgl.dataloading.negative_sampler.PerSourceUniform(5),
        )

        return dgl.dataloading.DataLoader(
            self.g_bid,
            self.train_nid,
            sampler,
            device=self.device,
            batch_size=self.batch_size,
            shuffle=True,
            drop_last=False,
            # num_workers=self.num_workers,
        )

    def val_dataloader(self):
        sampler = dgl.dataloading.as_edge_prediction_sampler(
            self.sampler,
            exclude="reverse_id",
            reverse_eids=self.reverse_eids,
            negative_sampler=NegativeSampler(self.g, 1)
            # negative_sampler=dgl.dataloading.negative_sampler.PerSourceUniform(5),
        )

        return dgl.dataloading.DataLoader(
            self.g_bid,
            self.val_nid,
            sampler,
            device=self.device,
            batch_size=self.batch_size,
            shuffle=False,
            drop_last=False,
            # num_workers=self.num_workers,
        )


In [11]:
# Exploring DataModule class
csv_dataset_root = '../' + cfg.data.zillow_root
force_reload=False
dataset = dgl.data.CSVDataset(csv_dataset_root, force_reload=force_reload)
g = dataset[0]

Done loading data from cached files.


Graph(num_nodes=84262, num_edges=61498,
      ndata_schemes={'feat': Scheme(shape=(512,), dtype=torch.float32), 'test_mask': Scheme(shape=(), dtype=torch.uint8), 'val_mask': Scheme(shape=(), dtype=torch.uint8), 'train_mask': Scheme(shape=(), dtype=torch.uint8)}
      edata_schemes={})

## Step 2: Exploring SAGELightning Class

This is our model that we will train on graph data

In [11]:
class ScorePredictor(nn.Module):
    def forward(self, edge_subgraph, x):
        with edge_subgraph.local_scope():
            edge_subgraph.ndata["h"] = x
            edge_subgraph.apply_edges(fn.u_dot_v("h", "h", "score"))
            return edge_subgraph.edata["score"]


class SAGELightning(LightningModule):
    def __init__(
        self,
        in_dim,
        h_dim,
        n_layers=3,
        activation=F.relu,
        dropout=0,
        sage_conv_method="mean",
        lr=0.0005,
        batch_size=1024,
    ):
        super().__init__()
        self.module = SAGE(
            in_dim, h_dim, n_layers, activation, dropout, sage_conv_method
        )
        self.lr = lr
        self.predictor = ScorePredictor()
        self.batch_size = batch_size
        self.save_hyperparameters() # specify that the module should save hyperparameters on each training loop

        self.train_loss = MeanMetric() # aggregate loss per batch using mean
        self.val_positive_distance = MeanMetric()
        self.val_negative_distance = MeanMetric()

    def forward(self, graph, blocks, x):
        self.module(graph, blocks, x) # Just runs SAGE() model defined in SAGE.py

    def training_step(self, batch, batch_idx):
        input_nodes, pos_graph, neg_graph, blocks = batch
        x = blocks[0].srcdata["feat"]
        logits = self.module(blocks, x)
        pos_score = self.predictor(pos_graph, logits)
        neg_score = self.predictor(neg_graph, logits)

        score = torch.cat([pos_score, neg_score])
        pos_label = torch.ones_like(pos_score)
        neg_label = torch.zeros_like(neg_score)
        labels = torch.cat([pos_label, neg_label])
        loss = F.binary_cross_entropy_with_logits(score, labels)
        self.train_loss(loss)
        self.log(
            "train_loss",
            self.train_loss,
            prog_bar=True,
            on_step=True,
            on_epoch=False,
            batch_size=self.batch_size,
        )

        return loss

    def validation_step(self, batch, batch_idx):
        input_nodes, pos_graph, neg_graph, blocks = batch
        x = blocks[0].srcdata["feat"]
        logits = self.module(blocks, x)
        pos_score = self.predictor(pos_graph, logits)
        neg_score = self.predictor(neg_graph, logits)

        self.val_positive_distance(pos_score)
        self.val_negative_distance(neg_score)

        self.log(
            "mean_val_posititve_distance",
            self.val_positive_distance,
            prog_bar=True,
            on_step=False,
            on_epoch=True,
            batch_size=self.batch_size,
        )
        self.log(
            "mean_val_negative_distance",
            self.val_negative_distance,
            prog_bar=True,
            on_step=False,
            on_epoch=True,
            batch_size=self.batch_size,
        )

    def configure_optimizers(self):
        optimizer = torch.optim.Adam(self.parameters(), lr=self.lr)
        return optimizer

In [4]:
if not torch.cuda.is_available():
    device = "cpu"
else:
    device = "cuda"

In [22]:
datamodule = DataModule(
    '../'+cfg.data.zillow_root, device=device, batch_size=cfg.training.batch_size
)

model = SAGELightning(
    datamodule.in_dim,
    cfg.model.hidden_dim,
    n_layers=cfg.model.n_layers,
    batch_size=cfg.training.batch_size,
)

checkpoint_callback = ModelCheckpoint(
    monitor="mean_val_negative_distance", save_top_k=1, mode="max"
)
trainer = Trainer(accelerator="gpu", max_epochs=1, callbacks=[checkpoint_callback])

trainer.fit(model, datamodule=datamodule)

Done loading data from cached files.


GPU available: True (cuda), used: True
TPU available: False, using: 0 TPU cores
IPU available: False, using: 0 IPUs
HPU available: False, using: 0 HPUs
LOCAL_RANK: 0 - CUDA_VISIBLE_DEVICES: [0]

  | Name                  | Type           | Params
---------------------------------------------------------
0 | module                | SAGE           | 525 K 
1 | predictor             | ScorePredictor | 0     
2 | train_loss            | MeanMetric     | 0     
3 | val_positive_distance | MeanMetric     | 0     
4 | val_negative_distance | MeanMetric     | 0     
---------------------------------------------------------
525 K     Trainable params
0         Non-trainable params
525 K     Total params
2.100     Total estimated model params size (MB)


Sanity Checking DataLoader 0:   0%|          | 0/2 [00:00<?, ?it/s]

/ext3/conda/zillow_MMKG/lib/python3.8/site-packages/pytorch_lightning/utilities/data.py:135: UserWarning: Your `IterableDataset` has `__len__` defined. In combination with multi-process data loading (when num_workers > 1), `__len__` could be inaccurate if each worker is not configured independently to avoid having duplicate data.
  rank_zero_warn(
/ext3/conda/zillow_MMKG/lib/python3.8/site-packages/pytorch_lightning/trainer/connectors/data_connector.py:224: PossibleUserWarning: The dataloader, val_dataloader 0, does not have many workers which may be a bottleneck. Consider increasing the value of the `num_workers` argument` (try 48 which is the number of cpus on this machine) in the `DataLoader` init to improve performance.
  rank_zero_warn(
/ext3/conda/zillow_MMKG/lib/python3.8/site-packages/lightning_lite/utilities/data.py:63: UserWarning: Your `IterableDataset` has `__len__` defined. In combination with multi-process data loading (when num_workers > 1), `__len__` could be inaccurate

/ext3/conda/zillow_MMKG/lib/python3.8/site-packages/pytorch_lightning/trainer/connectors/data_connector.py:224: PossibleUserWarning: The dataloader, train_dataloader, does not have many workers which may be a bottleneck. Consider increasing the value of the `num_workers` argument` (try 48 which is the number of cpus on this machine) in the `DataLoader` init to improve performance.
  rank_zero_warn(


Epoch 0: 100%|██████████| 71/71 [00:01<00:00, 67.42it/s, loss=0.693, v_num=5, train_loss=0.693, mean_val_posititve_distance=0.00551, mean_val_negative_distance=0.000519]

`Trainer.fit` stopped: `max_epochs=1` reached.


Epoch 0: 100%|██████████| 71/71 [00:01<00:00, 66.38it/s, loss=0.693, v_num=5, train_loss=0.693, mean_val_posititve_distance=0.00551, mean_val_negative_distance=0.000519]


In [5]:
modal_node_ids_file = '../'+cfg.data.zillow_root+'/modal_node_ids.json'
datamodule = DataModule(
    '../'+cfg.data.zillow_root, modal_node_ids_file, device=device, batch_size=cfg.training.batch_size
)
train_dataloader = datamodule.train_dataloader()

Done loading data from cached files.


In [6]:
i = 0
end = 58
max_img_id = max(modal_node_ids['images'])
min_kw_id = min(modal_node_ids['keywords'])

same_node_connect_pos_subgraph = []
same_node_connect_neg_subgraph = []

for batch in train_dataloader:
    input_nodes, pos_graph, neg_graph, blocks = batch
    pos_mapper = dict(zip(pos_graph.nodes().detach().cpu().numpy(), 
                pos_graph.ndata['_ID'].detach().cpu().numpy()))

    neg_mapper = dict(zip(neg_graph.nodes().detach().cpu().numpy(), 
                neg_graph.ndata['_ID'].detach().cpu().numpy()))

    src_pos, dst_pos = [x.detach().cpu().numpy() for x in pos_graph.edges()]
    src_neg, dst_neg = [x.detach().cpu().numpy() for x in neg_graph.edges()]
    
    src_pos = [pos_mapper[src] for src in src_pos]
    dst_pos = [pos_mapper[dst] for dst in dst_pos]

    src_neg = [neg_mapper[src] for src in src_neg]
    dst_neg = [neg_mapper[dst] for dst in dst_neg]

    src_dst_pos, src_dst_neg = list(zip(src_pos, dst_pos)), list(zip(src_neg, dst_neg))

    for pair in src_dst_pos:
        if (pair[0]<max_img_id and pair[1]<max_img_id) or (pair[0]>min_kw_id and pair[1]>min_kw_id):
            same_node_connect_pos_subgraph.append(pair)
    
    for pair in src_dst_neg:
        if (pair[0]<max_img_id and pair[1]<max_img_id) or (pair[0]>min_kw_id and pair[1]>min_kw_id):
            same_node_connect_neg_subgraph.append(pair)
    
    i += 1
    if i == end:
        break

print(len(same_node_connect_pos_subgraph))
print(len(same_node_connect_neg_subgraph))
print(len(src_dst_pos))
print(len(src_dst_neg))

0
0
640
2330


In [34]:
print(src_dst_neg[:10])

[(1212, 83135), (1212, 82755), (1212, 83069), (1212, 82817), (1212, 82992), (3595, 82890), (3595, 83239), (3595, 82731), (3595, 82775), (3595, 82929)]


In [9]:
print(neg_graph)

Graph(num_nodes=2018, num_edges=5120,
      ndata_schemes={'_ID': Scheme(shape=(), dtype=torch.int64)}
      edata_schemes={})
